## 5.基于Xgboost分类算法的电商平台用户购买意向预测 

**1.数据预处理**

**1.1.读取并展示数据**

导入pandas库，读取用户表以及2、3、4月用户行为数据

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 10)
user = pd.read_csv('../data/data_用户表.csv',engine='python',encoding='utf-8')#读取用户表
month2 = pd.read_csv('../data/data_2月用户行为数据.csv',engine='python',encoding='utf-8')#读取2月用户行为数据集
month3 = pd.read_csv('../data/data_3月用户行为数据.csv',engine='python',encoding='utf-8')#读取3月用户行为数据集
month4 = pd.read_csv('../data/data_4月用户行为数据.csv',engine='python',encoding='utf-8')#读取4月用户行为数据集

In [2]:
user 

,用户ID,年龄,性别,用户级别,用户注册时间
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26
...,...,...,...,...,...
105316,305317,3.0,2.0,4,2013-04-10
105317,305318,4.0,2.0,3,2016-01-26
105318,305319,3.0,0.0,3,2016-01-26
105319,305320,4.0,2.0,2,2016-01-26


上诉结果解释：年龄为1代表15岁以下，2代表26-35岁，3代表36-45岁，4代表46-55岁，5代表56-65岁，6代表66岁以上，-1代表缺失值。

展示2月用户行为数据

In [3]:
month2

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌
0,217567,22387,2016-01-31 23:59:54,1,8,30
1,217567,22387,2016-01-31 23:59:54,6,8,30
2,207684,4732,2016-01-31 23:59:59,1,6,375
3,207684,4732,2016-01-31 23:59:59,1,6,375
4,207684,4732,2016-01-31 23:59:59,1,6,375
...,...,...,...,...,...,...
11485379,278856,153390,2016-02-29 23:59:56,6,7,157
11485380,263603,114890,2016-02-29 23:59:56,1,9,640
11485381,263603,114890,2016-02-29 23:59:56,6,9,640
11485382,224475,5505,2016-02-29 23:59:57,6,8,214


上诉结果解释：行为类别为1代表浏览，2代表添加购物车，3代表删除购物车，4代表购买，5代表收藏，6代表点击。

**1.2.验证数据表参照完整性（用户行为-用户）**

验证用户行为数据表中的用户是否存在用户表中

In [4]:
temp_df1 = pd.DataFrame({'用户ID':['a','b','f']})#类比“用户数据集”
temp_df2 = pd.DataFrame({'用户ID':['a','a','e','c'],'用户行为类别':[1,1,2,3]})#类比“用户行为数据集”
temp_df3 = pd.merge(temp_df1,temp_df2)#使用merge函数，以两个表中“用户ID”为主键，将两个表连接起来，保留共有部分。
print(temp_df1)
print(temp_df2)
print(temp_df3)

  用户ID
0    a
1    b
2    f
  用户ID  用户行为类别
0    a       1
1    a       1
2    e       2
3    c       3
  用户ID  用户行为类别
0    a       1
1    a       1


In [5]:
user_id = user['用户ID'].to_frame()#取到“用户ID”列数据，to_frame：将数据转换为DataFrame格式

print('2月用户行为数据集中的用户是否在用户表中？',len(month2) == len(pd.merge(user_id,month2)))#判断2月用户行为数据集中的用户是否存在用户表中？
print('3月用户行为数据集中的用户是否在用户表中？',len(month3) == len(pd.merge(user_id,month3)))
print('4月用户行为数据集中的用户是否在用户表中？',len(month4) == len(pd.merge(user_id,month4)))

2月用户行为数据集中的用户是否在用户表中？ True
3月用户行为数据集中的用户是否在用户表中？ True
4月用户行为数据集中的用户是否在用户表中？ True


上诉结果解释：用户数据集中的用户与用户行为数据集中的用户完全一致。根据merge前后的数量比对，能保证用户行为数据集中的用户ID是用户表中的用户ID的子集。

**1.3.查看数据中重复记录（保留反复浏览）**

In [6]:
month2.duplicated()#查看3月用户行为数据集中的记录是否存在重复，True：代表重复

0           False
1           False
2           False
3            True
4            True
            ...  
11485379     True
11485380     True
11485381     True
11485382    False
11485383    False
Length: 11485384, dtype: bool

In [7]:
month2.duplicated().sum()

5754250

统计一下三月份重复数据的数量

In [8]:
count = 0#将变量count赋值为0
for item in month2.duplicated():
    if item == True:
        #统计重复数据的数量
        count =count + 1
count

5754250

In [9]:
count = 0 
for item in month3.duplicated():
    if item == True:
        
        count =count + 1
count

13703224

In [10]:
count = 0
for item in month4.duplicated():
    if item == True:
     
        count =count + 1
count

7262521

上诉结果解释：由上面输出的结果，可以看到有许多条重复，有两种解释：第一种：用户喜欢才重复操作；第二种：用户无意识的情况造成重复。可以采用两种方式来做：1）保留重复；2）去除重复。以下的做法是先保留，不去掉这个重复。

读取重复的数据行

In [11]:
IsDuplicated = month2.duplicated()#查看2月用户行为数据集中的记录是否存在重复，true：代表重复，将结果保存到IsDuplicated数组中。
month2_isDuplicated=month2[IsDuplicated]#获取重复数据，返回true的行，[]代表筛选满足条件的行

根据不同行为类别进行分组计数，查看造成重复数据的原因

In [12]:
month2_isDuplicated.groupby('行为类别').count()#根据“行为类型”进行分组统计

,用户ID,商品ID,时间,商品类别,商品品牌
行为类别,,,,,
1,2176378,2176378,2176378,2176378,2176378
2,636,636,636,636,636
3,1464,1464,1464,1464,1464
4,37,37,37,37,37
5,1981,1981,1981,1981,1981
6,3573754,3573754,3573754,3573754,3573754


上诉结果解释：发现重复数据大多数都是由于用户的行为类别：浏览（1），或者点击（6）产生。

**1.4.检查用户注册、行为时间是够正确**

检查是否存在注册时间在2016年4月15日之后的用户

In [13]:
user['用户注册时间']=pd.to_datetime(user['用户注册时间'])#将“用户注册时间”列取出来，并转换成datatime类型
user_after0415 = user.loc[user.用户注册时间  >= '2016-4-15']#取出“用户注册时间”在2016年4月15日之后的数据
user_after0415.size#查看数据的个数

135

上诉结果解释：135：代表存在135个4月15日之后注册的用户。如果在用户行为数据集中该用户在4月15日之后不存在行为数据，那么这些用户，并不需要删除；如果该用户在4月15日之后存在行为数据，那么这些用户就需要删除。

查看4月15日之后注册的用户是否存在行为数据

In [14]:
month4['时间'] = pd.to_datetime(month4['时间'])#在4月用户行为数据集将“时间”列取出来，并转换成datatime类型
month4.loc[month4.时间 >= '2016-4-16']#找采4月用户行为数据集中“时间”在4月15日之后的用户行为数据

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌


上诉结果解释：说明4月15日之后注册的用户，没有行为数据，所以这一批用户不需要删除。

**1.5.构建用户-行为-购买转化率视图**

根据用户表、行为表，统计不同用户的不同行为次数，进一步得出不同用户的不同行为-购买行为转化率。

1.5.1.展示得到上述用户-行为=购买转化率视图的原始数据：

In [15]:
user.head()#展示用户表的前5行数据

,用户ID,年龄,性别,用户级别,用户注册时间
0,200001,6.0,2.0,5,2016-01-26
1,200002,-1.0,0.0,1,2016-01-26
2,200003,4.0,1.0,4,2016-01-26
3,200004,-1.0,2.0,1,2016-01-26
4,200005,2.0,0.0,4,2016-01-26


In [16]:
month2.head()#展示2月用户行为数据的前5行数据

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌
0,217567,22387,2016-01-31 23:59:54,1,8,30
1,217567,22387,2016-01-31 23:59:54,6,8,30
2,207684,4732,2016-01-31 23:59:59,1,6,375
3,207684,4732,2016-01-31 23:59:59,1,6,375
4,207684,4732,2016-01-31 23:59:59,1,6,375


1.5.2展示最终用户-行为-购买转化率视图：

In [17]:
result_User_Action_ConversionRates = pd.read_csv("../data/result_User_Action_ConversionRates.csv")#读取最终的用户-行为-购买转换率视图
result_User_Action_ConversionRates

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
0,200001,6.0,2.0,5,212.0,22.0,13.0,1.0,0.0,414.0,0.045455,0.004717,0.002415,1.0
1,200002,-1.0,0.0,1,238.0,1.0,0.0,0.0,0.0,484.0,0.000000,0.000000,0.000000,NaN
2,200003,4.0,1.0,4,221.0,4.0,1.0,0.0,1.0,420.0,0.000000,0.000000,0.000000,0.0
3,200004,-1.0,2.0,1,52.0,0.0,0.0,0.0,0.0,61.0,NaN,0.000000,0.000000,NaN
4,200005,2.0,0.0,4,106.0,2.0,3.0,1.0,2.0,161.0,0.500000,0.009434,0.006211,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105316,305317,3.0,2.0,4,216.0,1.0,0.0,1.0,2.0,278.0,1.000000,0.004630,0.003597,0.5
105317,305318,4.0,2.0,3,58.0,3.0,0.0,1.0,0.0,88.0,0.333333,0.017241,0.011364,1.0
105318,305319,3.0,0.0,3,590.0,3.0,0.0,0.0,0.0,1189.0,0.000000,0.000000,0.000000,NaN
105319,305320,4.0,2.0,2,68.0,1.0,0.0,0.0,0.0,150.0,0.000000,0.000000,0.000000,NaN


1.5.3具体的过程如下：

定义文件名

In [18]:
ACTION_201602_FILE = "../data/data_2月用户行为数据.csv"
ACTION_201603_FILE = "../data/data_3月用户行为数据.csv"
ACTION_201604_FILE = "../data/data_4月用户行为数据.csv"
USER_FILE = "../data/data_用户表.csv"

In [19]:
import numpy as np
from collections import Counter

编写函数add_user_type_count,分别统计每个用户进行不同操作的次数

In [20]:
def add_user_type_count(group):
    
    behavior_type = group.行为类别.astype(int)#将传入参数group中“行为类别”列数据转为整形，将转换后的一维数据保存在behavior_type中
    
    #使用count函数，统计“行为类别”中各行为类别（1：浏览，2：加购，3：删除，4：购买，5：收藏，6：点击）的数量以字典的形式返回，并保存在type_cnt中
    type_cnt = Counter(behavior_type)#统计用户行为类别
    
    group['浏览数'] = type_cnt[1]#在group中加入“浏览数”列，将type_cnt中key为1的value，即1类行为的数量，传递到“浏览数”列
    group['加购物车数'] = type_cnt[2]#在group中加入“加购物车数”列，将type_cnt中key为2的value，即2类行为（加购）的数量，传递到“加购物车数”列
    group['删购物车数'] = type_cnt[3]#在group中加入“删购物车数”列，将type_cnt中key为3的value，即3类行为（删购）的数量，传递到“删购物车数”列
    group['购买数'] = type_cnt[4]#在group中加入“购买数”列，将type_cnt中key为4的value，即4类行为（购买）的数量，传递到“购买数”列
    group['收藏数'] = type_cnt[5]#在group中加入“收藏数”列，将type_cnt中key为5的value，即5类行为（收藏）的数量，传递到“收藏数”列
    group['点击数'] = type_cnt[6]#在group中加入“点击数”列，将type_cnt中key为6的value，即6类行为（点击）的数量，传递到“点击数”列
    
    #返回结果
    return group[['用户ID', '浏览数', '加购物车数','删购物车数', '购买数', '收藏数', '点击数']]

In [21]:
#loc[:1000,[‘用户ID’，‘行为类别’]]:使用loc函数，取到2月用户行为数据集中的‘用户ID’和‘行为类别’两列的1000条数据
#groupby.apply(add_user_type_count)：是先根据“用户ID”进行分组，然后对每个单独的组，进行add_user_type_count函数的操作
#as_index=False：表示带索引形式返回数据
month2_groupbyUserId=month2.loc[:1000, ['用户ID', '行为类别']].groupby(['用户ID'], as_index=False).apply(add_user_type_count)
month2_groupbyUserId.head()

,用户ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数
0,217567,1,0,0,0,1,2
1,217567,1,0,0,0,1,2
2,207684,4,0,0,0,0,0
3,207684,4,0,0,0,0,0
4,207684,4,0,0,0,0,0


In [22]:
#drop_Duplicates（‘用户ID’）：根据“用户ID”列删除重复项，默认保留第一个出现的“用户Id”的数据
month2_groupbyUserId_dropDuplicates=month2_groupbyUserId.drop_duplicates('用户ID')
month2_groupbyUserId_dropDuplicates.head()

,用户ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数
0,217567,1,0,0,0,1,2
2,207684,4,0,0,0,0,0
6,245959,1,0,0,0,0,3
9,272629,8,0,0,0,0,20
16,284046,2,0,0,0,0,1


展示groupby方法中的as_index=True或者as_index=False

In [23]:
temp_df = pd.DataFrame(data={'books':['bk1','bk1','bk1','bk2','bk2','bk3'], 'price': [12,12,12,15,15,17],'num':[2,1,1,4,2,2]})
temp_df

,books,price,num
0,bk1,12,2
1,bk1,12,1
2,bk1,12,1
3,bk2,15,4
4,bk2,15,2
5,bk3,17,2


In [24]:
temp_df.groupby('books',as_index=True).sum()

,price,num
books,,
bk1,36,4
bk2,30,6
bk3,17,2


In [25]:
temp_df.groupby('books',as_index=False).sum()

,books,price,num
0,bk1,36,4
1,bk2,30,6
2,bk3,17,2


编写get_user_action_data，分别读取用户的行为数据集，避免一次读取数据量过大导致的溢出内存错误，并调用add_user_type_count统计用户不同行为数据。

In [26]:
#由于用户行为数据量较大，一次性读入可能造成内存错误（Memory Error），因而可以使用pandas分块（chunk）读取
#由于用户行为数据量较大，避免数据量过大，导致的内存溢出错误，因而可以使用pandas的分块（chunk）读取
#根据自己的笔记本的性能调节chunk_size大小
def get_user_action_data(fname, chunk_size=20000):
    reader=pd.read_csv(fname,chunksize = chunk_size,engine='python',encoding='utf-8')#以utf-8编码的形式，分块读取“fname”指定的数据文件，保存到reader中。
    chunks = []
    for chunk in reader:
        chunk= chunk[['用户ID', '行为类别']]#读取“用户ID”、“行为类别”两个字段的数据
        chunks.append(chunk)#将读取的数据依次放在chunks中
        
    #将块拼接为pandas dataframe格式
    #ignore_index=True，将传递的顺序连接各数据块，重新分配实际索引的范围。
    chunks_concated = pd.concat(chunks, ignore_index=True)
    
    #调用add_user_type_count函数，分组统计每个用户进行不同操作的次数
    #按用户ID分组，对每一组进行统计，as_index=False表示为数据框生成索引号（第一列数据不是索引号）
    #在行为数据集中，一个用户可能有多条记录，按照用户ID把同一个用户的记录放在一起
    chunks_concated = chunks_concated.groupby(['用户ID'], as_index=False).apply(add_user_type_count)
    
    chunks_concated = chunks_concated.drop_duplicates('用户ID')#将重复的行丢弃
    return chunks_concated

执行merge_action_data函数

In [27]:
def merge_action_data():
    action = []#定义action空列表
    action.append(get_user_action_data(fname=ACTION_201602_FILE))#调用get_user_action_data函数，读取2月份用户行为数据，依次放入d_ac列表中
    action.append(get_user_action_data(fname=ACTION_201603_FILE))#调用get_user_action_data函数，读取3月份用户行为数据，依次放入d_ac列表中
    action.append(get_user_action_data(fname=ACTION_201604_FILE))#调用get_user_action_data函数，读取4月份用户行为数据，依次放入d_ac列表中
    
    #将2、3、4月用户行为数据进行拼接
    #ignore_index=True，按传递的顺序了解各数据，重新分配实际索引的范围
    action = pd.concat(action, ignore_index=True)
    
    action = action.groupby(['用户ID'], as_index=False).sum()#根据“用户ID”对2、3、4月用户不同行为的统计量求和
    
    action['购买加购转化率'] = action['购买数'] / action['加购物车数']#计算并添加“购买加购转化率”列
    action['购买浏览转化率'] = action['购买数'] / action['浏览数'] #计算并添加“购买浏览转化率”列
    action['购买点击转化率'] = action['购买数'] / action['点击数'] #计算并添加“购买点击化率”列
    action['购买收藏转化率'] = action['购买数'] / action['收藏数'] #计算并添加“购买收藏转化率”列
    #
    action.loc[action['购买加购转化率'] > 1., '购买加购转化率'] = 1.#将大于1的“购买加购转化率”字段设置为1（100%）,确保数据没有问题
    action.loc[action['购买浏览转化率'] > 1., '购买浏览转化率'] = 1.#将大于1的“购买加购转化率”字段设置为1（100%）,确保数据没有问题
    action.loc[action['购买点击转化率'] > 1., '购买点击转化率'] = 1.#将大于1的“购买加购转化率”字段设置为1（100%）,确保数据没有问题
    action.loc[action['购买收藏转化率'] > 1., '购买收藏转化率'] = 1.#将大于1的“购买加购转化率”字段设置为1（100%）,确保数据没有问题
    
    return action

执行merge_action_adta函数

In [28]:
action_merged = merge_action_data()#执行merge_action_data函数，将2、3、4月的数据合并到一起，并求各类转换率，保存user_behavior表
action_merged

,用户ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
0,200001,212,22,13,1,0,414,0.045455,0.004717,0.002415,1.0
1,200002,238,1,0,0,0,484,0.000000,0.000000,0.000000,NaN
2,200003,221,4,1,0,1,420,0.000000,0.000000,0.000000,0.0
3,200004,52,0,0,0,0,61,NaN,0.000000,0.000000,NaN
4,200005,106,2,3,1,2,161,0.500000,0.009434,0.006211,0.5
...,...,...,...,...,...,...,...,...,...,...,...
105175,305317,216,1,0,1,2,278,1.000000,0.004630,0.003597,0.5
105176,305318,58,3,0,1,0,88,0.333333,0.017241,0.011364,1.0
105177,305319,590,3,0,0,0,1189,0.000000,0.000000,0.000000,NaN
105178,305320,68,1,0,0,0,150,0.000000,0.000000,0.000000,NaN


In [29]:
user_need = user[["用户ID", "年龄", "性别", "用户级别"]]#从用户表中获取“用户ID”、“年龄”、“性别”、“用户级别”4列数据，保存为user_base表

连接成一张表并保存

In [30]:
user_action=pd.merge(user_need,action_merged,on=['用户ID'],how='left')
user_action.to_csv("../data/result_User_Action_ConversionRates.csv", index=False)

上诉结果解释：左联接（left jion）返回左表中的所有记录和右表中联接字段相等的记录

展示最终得到的用户-行为-购买转化率视图：

In [31]:
result_User_Action_ConversionRates = pd.read_csv("../data/result_User_Action_ConversionRates.csv")#展示最终得到的用户-行为-购买转化率视图
result_User_Action_ConversionRates

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
0,200001,6.0,2.0,5,212.0,22.0,13.0,1.0,0.0,414.0,0.045455,0.004717,0.002415,1.0
1,200002,-1.0,0.0,1,238.0,1.0,0.0,0.0,0.0,484.0,0.000000,0.000000,0.000000,NaN
2,200003,4.0,1.0,4,221.0,4.0,1.0,0.0,1.0,420.0,0.000000,0.000000,0.000000,0.0
3,200004,-1.0,2.0,1,52.0,0.0,0.0,0.0,0.0,61.0,NaN,0.000000,0.000000,NaN
4,200005,2.0,0.0,4,106.0,2.0,3.0,1.0,2.0,161.0,0.500000,0.009434,0.006211,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105316,305317,3.0,2.0,4,216.0,1.0,0.0,1.0,2.0,278.0,1.000000,0.004630,0.003597,0.5
105317,305318,4.0,2.0,3,58.0,3.0,0.0,1.0,0.0,88.0,0.333333,0.017241,0.011364,1.0
105318,305319,3.0,0.0,3,590.0,3.0,0.0,0.0,0.0,1189.0,0.000000,0.000000,0.000000,NaN
105319,305320,4.0,2.0,2,68.0,1.0,0.0,0.0,0.0,150.0,0.000000,0.000000,0.000000,NaN


**1.6.删除无交互行为的用户、惰性用户**

读取用户-行为-购买转化率视图，对各特征进行描述性统计分析。

In [32]:
rUACRs = pd.read_csv('../data/result_User_Action_ConversionRates.csv',header=0)#读取用户-行为-购买转化率视图
pd.options.display.float_format = '{:,.2f}'.format#输出格式设置，保留两位小数
rUACRs.describe()#展示统计指标

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
count,"105,321.00","105,318.00","105,318.00","105,321.00","105,180.00","105,180.00","105,180.00","105,180.00","105,180.00","105,180.00","72,129.00","105,172.00","103,197.00","45,986.00"
mean,"252,661.00",2.77,1.11,3.85,180.47,5.47,2.43,0.46,1.04,291.22,0.15,0.01,0.01,0.55
std,"30,403.70",1.67,0.96,1.07,273.44,10.62,5.60,1.05,3.44,460.03,0.27,0.02,0.07,0.47
min,"200,001.00",-1.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
25%,"226,331.00",3.00,0.00,3.00,40.00,0.00,0.00,0.00,0.00,59.00,0.00,0.00,0.00,0.00
50%,"252,661.00",3.00,2.00,4.00,94.00,2.00,0.00,0.00,0.00,148.00,0.00,0.00,0.00,1.00
75%,"278,991.00",4.00,2.00,5.00,212.00,6.00,3.00,1.00,0.00,342.00,0.17,0.00,0.00,1.00
max,"305,321.00",6.00,2.00,5.00,"7,605.00",369.00,231.00,50.00,99.00,"15,302.00",1.00,1.00,1.00,1.00


上述结果解释：用上诉统计信息发现：第一行中根据用户ID统计发现有105321个用户，发现有3个用户没有年龄、性别字段，而且根据浏览、加购、删购、购买等记录却只有1463条记录，说明存在用户无任何交互记录，因此可以删除上诉用户。

In [33]:
rUACRs[rUACRs['年龄'].isnull()]#读取年龄、性别字段为空的用户数据

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
34072,234073,NaN,NaN,1,32.00,6.00,4.00,1.00,0.00,41.00,0.17,0.03,0.02,1.00
38905,238906,NaN,NaN,1,171.00,3.00,2.00,2.00,3.00,464.00,0.67,0.01,0.00,0.67
67704,267705,NaN,NaN,1,342.00,18.00,8.00,0.00,0.00,743.00,0.00,0.00,0.00,NaN


In [34]:
indices_isnull = rUACRs[rUACRs['年龄'].isnull()].index#读取“年龄”字段为空的数据的索引
rUACRs.drop(indices_isnull,axis=0,inplace=True)#在原表中删除“年龄”字段为空的数据

展示无交互行为（浏览、加购、删购、购买、收藏、点击）的用户数据

In [35]:
#展示无交互行为（浏览、加购、删购、购买、收藏、点击）的用户数据
no_action = rUACRs[(rUACRs['浏览数'].isnull()) & (rUACRs['加购物车数'].isnull()) & (rUACRs['删购物车数'].isnull()) & 
                   (rUACRs['购买数'].isnull()) & (rUACRs['收藏数'].isnull()) & (rUACRs['点击数'].isnull())]
no_action

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
1692,201693,-1.00,2.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936,201937,3.00,2.00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2273,202274,-1.00,2.00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2867,202868,3.00,2.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3530,203531,3.00,0.00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104309,304310,3.00,2.00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104636,304637,4.00,2.00,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104770,304771,3.00,0.00,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104981,304982,3.00,2.00,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


删除无交互行为（浏览、加购、删购、购买、收藏、点击）的用户

In [36]:
rUACRs.drop(no_action.index,axis=0,inplace=True)
print (len(rUACRs))#显示有交互行为的用户数量

105177


统计无购买记录的用户数量

In [37]:
no_buy = rUACRs[rUACRs['购买数']==0]#获取无购买记录的用户数据
print (len(no_buy))#输出购买数为0的用户数量

75694


统计有购买记录的用户数量

In [38]:
rUACRs = rUACRs[rUACRs['购买数']!=0]#获取有购买记录的用户数据
print (len(rUACRs))#输出购买数不为0的用户数量

29483


In [39]:
rUACRs.describe()

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
count,"29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00","29,483.00"
mean,"250,746.44",2.91,1.02,4.27,302.49,10.52,4.67,1.64,1.68,486.65,0.36,0.02,0.03,0.86
std,"29,979.68",1.49,0.96,0.81,391.53,14.30,7.57,1.41,4.58,658.67,0.32,0.04,0.14,0.29
min,"200,001.00",-1.00,0.00,2.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.01
25%,"225,058.50",3.00,0.00,4.00,76.00,3.00,0.00,1.00,0.00,116.00,0.12,0.00,0.00,1.00
50%,"249,144.00",3.00,1.00,4.00,178.00,6.00,2.00,1.00,0.00,282.00,0.25,0.01,0.01,1.00
75%,"276,252.50",4.00,2.00,5.00,381.00,13.00,6.00,2.00,1.00,604.00,0.50,0.02,0.01,1.00
max,"305,318.00",6.00,2.00,5.00,"7,605.00",288.00,178.00,50.00,96.00,"15,302.00",1.00,1.00,1.00,1.00


上诉结果解释：由上表所知，购买浏览数转化率和构面点击转化率均值分别为0.02和0.03

删除惰性用户。这里认为“浏览购买转化率”和“点击购买转化率”小于0.0005的用户为惰性用户

删除“浏览购买转化率”小于0.0005的用户

In [40]:
indices_little_Buy = rUACRs[rUACRs['购买浏览转化率']<0.0005].index#获取“浏览购买转化率”小于0.0005的用户数据索引
print (len(indices_little_Buy))#获取“浏览购买转化率”小于0.0005的用户数量
rUACRs.drop(indices_little_Buy,axis=0,inplace=True)#输出“浏览购买转化率”小于0.0005的用户

90


删除“点击购买转化率”小于0.0005的用户

In [41]:
indices_little_ConversionRates = rUACRs[rUACRs['购买点击转化率']<0.0005].index#获取“点击购买转化率”小于0.0005的用户数据索引
print (len(indices_little_ConversionRates))#获取“点击购买转化率”小于0.0005的用户数量
rUACRs.drop(indices_little_ConversionRates,axis=0,inplace=True)#输出“点击购买转化率”小于0.0005的用户

323


In [42]:
rUACRs.describe()

,用户ID,年龄,性别,用户级别,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率
count,"29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00","29,070.00"
mean,"250,767.10",2.91,1.03,4.27,280.26,10.14,4.46,1.64,1.59,447.11,0.36,0.02,0.03,0.87
std,"29,998.87",1.49,0.96,0.81,325.13,13.44,7.00,1.42,4.29,530.99,0.32,0.04,0.14,0.28
min,"200,001.00",-1.00,0.00,2.00,1.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.02
25%,"225,036.00",3.00,0.00,4.00,75.00,3.00,0.00,1.00,0.00,114.00,0.12,0.00,0.00,1.00
50%,"249,200.50",3.00,1.00,4.00,174.00,6.00,2.00,1.00,0.00,275.00,0.25,0.01,0.01,1.00
75%,"276,284.00",4.00,2.00,5.00,366.00,13.00,6.00,2.00,1.00,585.00,0.50,0.02,0.01,1.00
max,"305,318.00",6.00,2.00,5.00,"5,007.00",288.00,158.00,50.00,69.00,"8,156.00",1.00,1.00,1.00,1.00


上诉结果解释：最后这29070个用户为最终预测用户数据集

**1.7.构建商品-行为-购买转化率视图**

根据商品表、评论表、行为表，统计不同商品被执行不同行为次数，进一步得出不同商品的不同行为-购买行为的转化率。

展示原始数据

In [43]:
month2#展示2月用户行为数据集

,用户ID,商品ID,时间,行为类别,商品类别,商品品牌
0,217567,22387,2016-01-31 23:59:54,1,8,30
1,217567,22387,2016-01-31 23:59:54,6,8,30
2,207684,4732,2016-01-31 23:59:59,1,6,375
3,207684,4732,2016-01-31 23:59:59,1,6,375
4,207684,4732,2016-01-31 23:59:59,1,6,375
...,...,...,...,...,...,...
11485379,278856,153390,2016-02-29 23:59:56,6,7,157
11485380,263603,114890,2016-02-29 23:59:56,1,9,640
11485381,263603,114890,2016-02-29 23:59:56,6,9,640
11485382,224475,5505,2016-02-29 23:59:57,6,8,214


In [44]:
product = pd.read_csv('../data/data_商品表.csv',engine='python',encoding='utf-8')#读取商品表数据
product#展示商品表数据

,商品ID,属性1,属性2,属性3,商品类别,商品品牌
0,100141,3,1,2,8,214
1,10015,2,1,1,8,124
2,100150,3,-1,-1,8,214
3,100152,3,1,-1,8,306
4,100154,3,1,1,8,489
...,...,...,...,...,...,...
24142,99844,3,1,1,8,489
24143,99853,3,2,2,8,489
24144,99859,3,2,2,8,489
24145,99862,2,2,2,8,812


In [45]:
comment = pd.read_csv('../data/data_评论表.csv',engine='python',encoding='utf-8')#读取评论表数据
comment

,评论时间,商品ID,评论数量,存在差评,差评率
0,2016-02-01,100087,4,1,0.04
1,2016-02-01,100088,2,0,0.00
2,2016-02-01,100097,3,1,0.09
3,2016-02-01,1001,2,0,0.00
4,2016-02-01,100100,4,1,0.06
...,...,...,...,...,...
558507,2016-04-15,99905,4,1,0.03
558508,2016-04-15,99906,4,1,0.04
558509,2016-04-15,99909,3,1,0.05
558510,2016-04-15,9991,4,1,0.05


上诉结果解释：评论数量字段：0代表无评论，1代表1条评论，2代表2-10条评论，3代表11-50条评论，4代表大于50条评论。存在差评字段：1为存在差评，0为不存在差评。

1.7.2展示最终的商品-行为-购买转化率视图

In [46]:
result_Item_Action_ConversionRates = pd.read_csv('../data/result_Item_Action_ConversionRates.csv',engine='python',encoding='utf-8')
#展示最终得到的商品-行为-购买转化率视图
result_Item_Action_ConversionRates

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
0,100174,2,2,1,8,812,249.00,5.00,2.00,0.00,2.00,350.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
1,100310,3,2,-1,8,214,126.00,1.00,0.00,0.00,1.00,266.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
2,100344,3,1,1,8,214,0.00,0.00,0.00,0.00,0.00,10.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN
3,100462,3,1,2,8,214,860.00,21.00,5.00,0.00,5.00,"1,689.00",0.00,0.00,0.00,0.00,3.00,0.00,0.00
4,100466,3,1,1,8,306,4.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,4.00,1.00,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3925,99625,3,1,1,8,214,"1,999.00",75.00,30.00,7.00,14.00,"3,416.00",0.09,0.00,0.00,0.50,3.00,0.00,0.00
3926,99639,3,1,1,8,214,1.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,3.00,1.00,0.07
3927,99646,1,2,1,8,30,"4,335.00",146.00,61.00,9.00,21.00,"6,714.00",0.06,0.00,0.00,0.43,4.00,1.00,0.03
3928,99774,1,1,2,8,403,12.00,0.00,0.00,0.00,0.00,13.00,NaN,0.00,0.00,NaN,4.00,1.00,0.05


1.7.3具体的实现过程如下：

定义文件名

In [47]:
COMMENT_FILE = "../data/data_评论表.csv"
PRODUCT_FILE = "../data/data_商品表.csv"

读取商品表中的商品数据

In [48]:
#定义get_data_product函数，读取商品表数据
def get_product_data():
    
    d_item=pd.read_csv(PRODUCT_FILE,header=0,engine='python',encoding='utf-8')#读取商品表的数据
    
    return d_item

编写函数add_product_type_count，分组统计每个商品被不同用户执行各类操作的次数

In [49]:
def add_product_type_count(group):#定义add_product_type_count函数
    behavior_type = group.行为类别.astype(int)#将传入参数group中“行为类别”列数据转为整形，将转换后的一维数组保存在behavior_type中
    
    #使用Counter函数，统计“行为类别”中各行为类别（1：浏览，2：加购，3：删除，4：购买，5：收藏，6：点击）的数量以字典的形式返回，并保存在type_cnt中
    type_cnt = Counter(behavior_type)
     
    group['浏览数'] = type_cnt[1]
    group['加购物车数'] = type_cnt[2]
    group['删购物车数'] = type_cnt[3]
    group['购买数'] = type_cnt[4]
    group['收藏数'] = type_cnt[5]
    group['点击数'] = type_cnt[6]
    
    return group[['商品ID', '浏览数', '加购物车数', '删购物车数', '购买数', '收藏数', '点击数']]

In [50]:
#举一个add_product_type_count函数应用的例子，选取2月份前1000行数据，分组统计每个商品被执行不同操作的次数
month2.loc[:1000, ['商品ID', '行为类别']].groupby(['商品ID'], as_index=False).apply(add_product_type_count).drop_duplicates('商品ID')

,商品ID,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数
0,22387,1,0,0,0,0,1
2,4732,29,0,0,0,0,33
6,145895,1,0,0,0,0,3
9,107774,8,0,0,0,0,9
16,87598,2,0,0,0,0,1
...,...,...,...,...,...,...,...
969,83325,2,0,0,0,0,2
980,133005,2,0,0,0,0,2
981,134683,2,0,0,0,0,2
998,55092,0,0,0,0,1,0


编写函数get_product_action_data，分块读取用户数据集中的数据。

In [51]:
#对用户行为数据表中的数据进行统计
def get_product_action_data(fname, chunk_size=20000):
    reader = pd.read_csv(fname, header=0, iterator=True,engine='python',encoding='utf-8')
    chunks = []
    loop = True
    while loop:
        try:
            #在这个循环中，每次只读一小段，只读“商品ID”和“行为类别”两个字段
            chunk = reader.get_chunk(chunk_size)[["商品ID", "行为类别"]]
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped")
    
    chunks_concated = pd.concat(chunks, ignore_index=True)#将块拼接为pandas dataframe格式
    #调用add_product_type_count函数，分组统计每个商品被执行不同操作的次数
    #按商品ID分组，对每一组进行统计，as_index =False表示无索引形式返回数据
    #在行为数据集中，一个用户可能有多条记录，按照用户Id把同一个用户的记录放在一起
    chunks_concated = chunks_concated.groupby(['商品ID'], as_index=False).apply(add_product_type_count)
    
    chunks_concated = chunks_concated.drop_duplicates('商品ID')#将重复的行丢弃
    return chunks_concated

编写get_comment_data函数，获取评论表中的数据。

In [52]:
#编写et_comment_data函数，获取评论表中的数据
def get_comment_data():
    comment = pd.read_csv(COMMENT_FILE, header=0,engine='python',encoding='utf-8')#读取评论表中的数据
    comment['评论时间'] = pd.to_datetime(comment['评论时间'])#将“评论时间”转换为dataframe格式
    
    #获取评论表中的数据
    #使用groupby函数，根据商品Id进行分组
    #如果存在某一个商品有多个日期的评论，则去时间较晚的那一个
    #使用transform（max）判断“商品ID”对应的“评论时间”是否为较晚的“评论时间”。返回True或False，保存在product_last_comment_indices中
    product_last_comment_indices = comment.groupby(['商品ID'])['评论时间'].transform(max) == comment['评论时间']
    
    comment = comment[product_last_comment_indices]#获取评论表中，“商品ID”对应的较晚的“评论时间”的数据
    #返回结果
    return comment[['商品ID', '评论数量','存在差评', '差评率']]

编写函数merge_action_data，合并2、3、4月数据，并计算各类转化率。

In [53]:
#编写函数merge_action_data,合并分块读取的数据，并计算各类转化率
def merge_action_data():
    action = []
    action.append(get_product_action_data(fname=ACTION_201602_FILE))
    action.append(get_product_action_data(fname=ACTION_201603_FILE))
    action.append(get_product_action_data(fname=ACTION_201604_FILE))
    
    #将块拼接为pandas dataframe格式
    #ignore= True，按传递的顺序连接各数据块，重新分配实际索引的范围。
    action = pd.concat(action, ignore_index=True)
    
    #按照商品ID，进行分组统计
    action = action.groupby(['商品ID'], as_index=False).sum()
    
    #构造转化率字段，转化率就是对购买结果的影响因素
    action['购买加购转化率'] = action['购买数'] / action['加购物车数']
    action['购买浏览转化率'] = action['购买数'] / action['浏览数']
    action['购买点击转化率'] = action['购买数'] / action['点击数']
    action['购买收藏转化率'] = action['购买数'] / action['收藏数']
    
    #将大于1的转化率字段设置为1（100%），确保数据没有问题
    action.loc[action['购买加购转化率'] > 1., '购买加购转化率'] = 1.
    action.loc[action['购买浏览转化率'] > 1., '购买浏览转化率'] = 1.
    action.loc[action['购买点击转化率'] > 1., '购买点击转化率'] = 1.
    action.loc[action['购买收藏转化率'] > 1., '购买收藏转化率'] = 1.
    
    return action

调用上诉函数，获取相关数据生成商品-行为-购买转化率视图。

In [54]:
#从商品表、行为表、评论表中获取数据
product = get_product_data()
action = merge_action_data()
comment = get_comment_data()

#依次将两张表连接在一张表，类似于SQL的左连接（left jion），on：指定住建
item_behavior = pd.merge( product, action, on=['商品ID'], how='left')
item_behavior = pd.merge( item_behavior, comment, on=['商品ID'], how='left')

item_behavior.to_csv("../data/result_Item_Action_ConversionRates.csv", index=False)#将最终连接后的表保存为商品-行为-转化率视图

Iteration is stopped
Iteration is stopped
Iteration is stopped


展示无交互行为（浏览、加购、删购、购买、收藏、点击）的商品数据

In [55]:
#展示无交互行为（浏览、加购、删购、购买、收藏、点击）的商品数据
no_action_product = item_behavior[(item_behavior['浏览数'].isnull()) & (item_behavior['加购物车数'].isnull()) & (item_behavior['删购物车数'].isnull()) & 
                   (item_behavior['购买数'].isnull()) & (item_behavior['收藏数'].isnull()) & (item_behavior['点击数'].isnull())]
no_action_product 

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
0,100141,3,1,2,8,214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10015,2,1,1,8,124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100150,3,-1,-1,8,214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100152,3,1,-1,8,306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.00,0.00,0.00
4,100154,3,1,1,8,489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24142,99844,3,1,1,8,489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24143,99853,3,2,2,8,489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24144,99859,3,2,2,8,489,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24145,99862,2,2,2,8,812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


删除无交互行为（浏览、加购、删购、购买、收藏、点击）的商品

In [56]:
item_behavior.drop(no_action_product.index,axis=0,inplace=True)#删除无交互行为（浏览、加购、删购、购买、收藏、点击）的商品
print (len(item_behavior))#展示有交互行为的商品数据

3930


In [57]:
item_behavior.describe()

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
count,"3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","3,930.00","2,221.00","3,901.00","3,684.00","2,013.00","2,817.00","2,817.00","2,817.00"
mean,"85,326.94",1.99,1.02,1.30,8.00,447.12,"1,727.08",54.32,21.33,3.38,10.68,"2,795.58",0.04,0.00,0.00,0.16,2.98,0.58,0.04
std,"49,296.10",1.15,0.91,0.91,0.00,252.53,"7,965.40",286.00,106.20,21.72,50.00,"12,659.80",0.09,0.00,0.00,0.26,1.04,0.49,0.10
min,18.00,-1.00,-1.00,-1.00,8.00,30.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00
25%,"42,680.50",1.00,1.00,1.00,8.00,214.00,19.00,0.00,0.00,0.00,0.00,30.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
50%,"85,920.50",2.00,1.00,2.00,8.00,403.00,148.00,1.00,1.00,0.00,1.00,247.00,0.00,0.00,0.00,0.00,3.00,1.00,0.02
75%,"127,364.00",3.00,2.00,2.00,8.00,658.00,658.00,12.00,5.00,0.00,4.00,"1,078.75",0.05,0.00,0.00,0.25,4.00,1.00,0.05
max,"171,182.00",3.00,2.00,2.00,8.00,916.00,"194,920.00","6,296.00","2,258.00",691.00,"1,205.00","312,005.00",1.00,0.11,0.06,1.00,4.00,1.00,1.00


展示最终得到的商品-行为-购买转化率视图：

In [58]:
item_behavior.to_csv("../data/result_Item_Action_ConversionRates.csv", index=False)#将最终连接后的表保存为商品-行为-购买转化率视图
item_behavior

,商品ID,属性1,属性2,属性3,商品类别,商品品牌,浏览数,加购物车数,删购物车数,购买数,收藏数,点击数,购买加购转化率,购买浏览转化率,购买点击转化率,购买收藏转化率,评论数量,存在差评,差评率
7,100174,2,2,1,8,812,249.00,5.00,2.00,0.00,2.00,350.00,0.00,0.00,0.00,0.00,2.00,0.00,0.00
20,100310,3,2,-1,8,214,126.00,1.00,0.00,0.00,1.00,266.00,0.00,0.00,0.00,0.00,NaN,NaN,NaN
25,100344,3,1,1,8,214,0.00,0.00,0.00,0.00,0.00,10.00,NaN,NaN,0.00,NaN,NaN,NaN,NaN
49,100462,3,1,2,8,214,860.00,21.00,5.00,0.00,5.00,"1,689.00",0.00,0.00,0.00,0.00,3.00,0.00,0.00
51,100466,3,1,1,8,306,4.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,4.00,1.00,0.02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24118,99625,3,1,1,8,214,"1,999.00",75.00,30.00,7.00,14.00,"3,416.00",0.09,0.00,0.00,0.50,3.00,0.00,0.00
24120,99639,3,1,1,8,214,1.00,0.00,0.00,0.00,0.00,0.00,NaN,0.00,NaN,NaN,3.00,1.00,0.07
24121,99646,1,2,1,8,30,"4,335.00",146.00,61.00,9.00,21.00,"6,714.00",0.06,0.00,0.00,0.43,4.00,1.00,0.03
24134,99774,1,1,2,8,403,12.00,0.00,0.00,0.00,0.00,13.00,NaN,0.00,0.00,NaN,4.00,1.00,0.05
